In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [2]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
!pip install -q datasets evaluate wandb transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-co

In [3]:
from datasets import load_dataset, get_dataset_config_names
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, classification_report
import evaluate
import wandb
import os
import random
import numpy as np
import torch

In [4]:
SEED = 42
NUM_PROC=5
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [5]:
TRANSFORMER="google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
class StatementDataset(torch.utils.data.Dataset):
    def __init__(self, statements, labels):
        self.statements = statements
        self.labels = labels

    def __getitem__(self, idx):
        encodings = tokenizer(self.statements[idx], truncation=True, padding=True)
        item = {key: torch.tensor(val) for key, val in encodings.items()}
        item['labels'] = int(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
dataset_base = 'mbzuai-ugrip-statement-tuning'
dataset_names=['Topic-Statements', 'sentiments', 'wikilingual_dataset', 'massive', 'exams', 'xlwic', 'paws-x', 'belebele']

In [10]:
# took 10 minutes to run - this will probably be replaced with a new dataset in the future, 2m if cached
tolerance = 20
# train : valid = 90:10 (800 -> 720:80): belebele is capped at 900
TRAIN_SIZE=800
TEST_SIZE=80

# let's just append the StatementDatasets to this big list
train_dataset = []
val_dataset = []

# this is to handle different naming conventions of different datasets
langs = {'english': ['en', 'eng_Latn', 'english'], 'chinese': ['zh', 'zho_Hans', 'chinese'], 'vietnamese': ['vi', 'vie_Latn', 'vietnamese'], 'french': ['fr', 'fra_Latn', 'french']}

for lang_key, lang_val in langs.items():
  for dataset_name in dataset_names:

    this_dataset_lang_codes = get_dataset_config_names(f"{dataset_base}/{dataset_name}")

    if len(this_dataset_lang_codes[0]) == 2:
      subset_lang = lang_val[0]
    elif "_" in this_dataset_lang_codes[0]:
      subset_lang = lang_val[1]
    # 'arabic', 'chinese', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'japanese', 'malay', 'portuguese', 'spanish'
    else:
      subset_lang = lang_val[2]

    # our language is unfortunately not available in this dataset, skip
    if subset_lang not in this_dataset_lang_codes:
      print(f"Skipping {lang_key} in {dataset_name}... because {this_dataset_lang_codes}")
      continue

    data = load_dataset(f"{dataset_base}/{dataset_name}", subset_lang)

    # eliminate column naming disparity
    if 'label' in data['train'].features and 'is_true' not in data['train'].features:
      data = data.rename_column('label', 'is_true')

    assert('label' not in data['train'].features)

    # example['is_true'] is not None AND len(tokenizer(example['statement']['input_ids']) < 512)
    train = data['train'].filter(lambda example: example["is_true"] is not None).filter(lambda example: len(tokenizer(example['statement'])['input_ids']) < 512)

    train = train.train_test_split(test_size=TRAIN_SIZE)['test']
    train_statements, val_statements, train_labels, val_labels = train_test_split(train['statement'], train['is_true'], test_size=TEST_SIZE, random_state=SEED, shuffle=True)

    this_train_dataset = StatementDataset(train_statements, train_labels)
    this_val_dataset = StatementDataset(val_statements, val_labels)

    train_dataset.append(this_train_dataset)
    val_dataset.append(this_val_dataset)

    print(f"Dataset: {dataset_name} Language: {lang_key}")
    print(f"Train size: {len(this_train_dataset)}")
    print(f"Test size: {len(this_val_dataset)}")

Dataset: Topic-Statements Language: english
Train size: 720
Test size: 80
Dataset: sentiments Language: english
Train size: 720
Test size: 80
Dataset: wikilingual_dataset Language: english
Train size: 720
Test size: 80
Dataset: massive Language: english
Train size: 720
Test size: 80
Skipping english in exams... because ['bg', 'hr', 'hu', 'it', 'mk', 'pl', 'pt', 'sq', 'sr', 'tr', 'vi']
Dataset: xlwic Language: english
Train size: 720
Test size: 80
Dataset: paws-x Language: english
Train size: 720
Test size: 80
Dataset: belebele Language: english
Train size: 720
Test size: 80
Dataset: Topic-Statements Language: chinese
Train size: 720
Test size: 80
Dataset: sentiments Language: chinese
Train size: 720
Test size: 80
Dataset: wikilingual_dataset Language: chinese
Train size: 720
Test size: 80
Dataset: massive Language: chinese
Train size: 720
Test size: 80
Skipping chinese in exams... because ['bg', 'hr', 'hu', 'it', 'mk', 'pl', 'pt', 'sq', 'sr', 'tr', 'vi']
Skipping chinese in xlwic... be

In [27]:
# this is a list of StatementDataset objects
print(train_dataset)

[<__main__.StatementDataset object at 0x7cd569e459f0>, <__main__.StatementDataset object at 0x7cd57ab5c130>, <__main__.StatementDataset object at 0x7cd580d31fc0>, <__main__.StatementDataset object at 0x7cd57bfcd630>, <__main__.StatementDataset object at 0x7cd5797cd0c0>, <__main__.StatementDataset object at 0x7cd580e50d00>, <__main__.StatementDataset object at 0x7cd57aaaef80>, <__main__.StatementDataset object at 0x7cd570602260>, <__main__.StatementDataset object at 0x7cd5702b9c60>, <__main__.StatementDataset object at 0x7cd58128e1d0>, <__main__.StatementDataset object at 0x7cd58119eb30>, <__main__.StatementDataset object at 0x7cd569e50e50>, <__main__.StatementDataset object at 0x7cd57025c0d0>, <__main__.StatementDataset object at 0x7cd569e0d8d0>, <__main__.StatementDataset object at 0x7cd5810adf60>, <__main__.StatementDataset object at 0x7cd56982c6d0>, <__main__.StatementDataset object at 0x7cd6c9a52740>, <__main__.StatementDataset object at 0x7cd580de5d50>, <__main__.StatementDataset 

In [11]:
# flatten
flat_train_statements = [statement for dataset in train_dataset for statement in dataset.statements]
flat_train_labels = [label for dataset in train_dataset for label in dataset.labels]

flat_val_statements = [statement for dataset in val_dataset for statement in dataset.statements]
flat_val_labels = [label for dataset in val_dataset for label in dataset.labels]

# Create a new dataset with the flattened lists
flat_train_dataset = StatementDataset(flat_train_statements, flat_train_labels)
flat_val_dataset = StatementDataset(flat_val_statements, flat_val_labels)

In [12]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries, which is in ids into text
    _, predictions = torch.max(torch.tensor(predictions), dim=1)


    return clf_metrics.compute(predictions=predictions, references=labels)

In [14]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [15]:
NUM_TRAIN_EPOCHS=6
TRAINING_BATCH=32
WARMUP=0.1
LEARNING_RATE=2e-06
EXPERIMENT_NAME="MBERT"
DECAY=0.01

In [18]:
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.2 MB/s eta 0:00:00


In [17]:
wandb.init(
    entity="mbzuai-ugrip",
    project=f"{EXPERIMENT_NAME}_multilingual_train",
    name=f"{EXPERIMENT_NAME}_{LEARNING_RATE}_{TRAINING_BATCH}_{WARMUP}_{DECAY}"
)

wandb: Currently logged in as: soph20 (mbzuai-ugrip). Use `wandb login --relogin` to force relogin


In [18]:
training_args = TrainingArguments(
    output_dir=f'./{EXPERIMENT_NAME}-outputs',          # output directory
    num_train_epochs=NUM_TRAIN_EPOCHS,              # total number of training epochs (num train epochs: 6)
    per_device_train_batch_size=TRAINING_BATCH,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_ratio=WARMUP,                # number of warmup steps for learning rate scheduler
    learning_rate=LEARNING_RATE,
    weight_decay=DECAY,               # strength of weight decay
    logging_dir=f'./{EXPERIMENT_NAME}-logs',            # directory for storing logs
    logging_steps=1000,
    save_steps=1000,
    evaluation_strategy='steps',
    save_total_limit=2,
    load_best_model_at_end= True,
    metric_for_best_model='f1',
    report_to="wandb",
)

model = AutoModelForSequenceClassification.from_pretrained(TRANSFORMER)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=flat_train_dataset,         # training dataset
    compute_metrics=compute_metrics,
    eval_dataset=flat_val_dataset,            # evaluation dataset
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub(f"sophiayk/m2-statement-tuning-mbert-multilingual-lr{LEARNING_RATE}-bs{TRAINING_BATCH}")